In [1]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [2]:
data, labels = preprocessing()
data['label'] = labels

X = data.iloc[:, :-1]
y = data['label']

In [3]:
from keras.utils import to_categorical
y = to_categorical(y)

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [5]:
IMG_SIZE = 48

X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(X_train[0][0][0])

X_train = X_train/255.0
X_test = X_test/255.0

print(X_train[0][0][0])
print(X_train.shape[1:])

[0]
[0.]
(48, 48, 1)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import time


dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:

#             NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
#             tensorboard = TensorBoard(log_dir=f"logs\{NAME}")

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X_train.shape[1:], activation='relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
                model.add(Dropout(0.2))

            model.add(Dense(10, activation='softmax'))

            model.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])

            model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20) # , callbacks=[tensorboard])
            
# model.save('6ax3-CNN.model')

Train on 8482 samples, validate on 4178 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
8482/8482 [==============================] - 5s 587us/sample - loss: 0.7284 - acc: 0.7386 - val_loss: 0.2414 - val_acc: 0.8935
Epoch 2/20
8482/8482 [==============================] - 3s 367us/sample - loss: 0.2141 - acc: 0.9151 - val_loss: 0.1291 - val_acc: 0.9493
Epoch 3/20
8482/8482 [==============================] - 3s 366us/sample - loss: 0.1396 - acc: 0.9466 - val_loss: 0.1062 - val_acc: 0.9588
Epoch 4/20
8482/8482 [==============================] - 3s 365us/sample - loss: 0.1062 - acc: 0.9610 - val_loss: 0.0856 - val_acc: 0.9667
Epoch 5/20
8482/8482 [==============================] - 3s 368us/sample - loss: 0.0841 - acc: 0.9677 - val_loss: 0.1081 - val_acc: 0.9557
Epoch 6/20
8482/8482 [==============================] - 3s 367us/sample - loss: 0.0662 - acc: 0.9759 - val_loss: 0.0721 - val_acc: 0.9725
Epoch 7/20
8482/8482 [==============================] - 3s 371us/sample - lo

In [ ]:
# model = tf.keras.models.load_model("6ax3-CNN.model")

test = 
prediction = model.predict('')